# FrameNet

I see three ways of getting features from FrameNet:

1. Does word $j$ evoke frame $i$?
2. Something with frame relations
3. Something with frame elements

In [1]:
import pandas as pd
from nltk.corpus import framenet as fn

### 1. Does word $j$ evoke frame $i$?

**In sum: too sparse to use.**

In [43]:
def get_lus(frame):
    """Helper to get lexemes from frame."""
    lus = list(frame['lexUnit'].keys())
    return [k.partition('.')[0] for k in lus]

In [116]:
all_frames = fn.frames('.*')
all_frame_names = [f.name for f in all_frames]
all_lus = [get_lus(f) for f in all_frames]
all_lus = [item for sublist in all_lus for item in sublist]
all_lus = list(set(all_lus))

In [125]:
evoke = pd.DataFrame(0, index=all_frame_names, columns=all_lus)
for frame in all_frames:
    name = frame.name
    lus = get_lus(frame)
    for lu in lus:
        evoke[lu][name] += 1

Most words evoke one frame, some two, few three.

In [128]:
evoke.max().value_counts()

1    9014
2     402
3       5
dtype: int64

In [130]:
evoke.head()

,blast,crimson,desist,major general,desolate,sandspit,afterwards,seat,slog,cajole,...,pastry,pasha,amass,technician,shall,rill,in front (of),satisfied,lavatory,hem
Abandonment,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Abounding_with,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Absorb_heat,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Abundance,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Abusing,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Frame relations

In [169]:
f = all_frames[4]
relations = f.frameRelations[0]
relations.keys()

dict_keys(['subID', 'supID', 'subFrameName', 'superFrameName', 'ID', '_type', 'feRelations', 'type', 'superFrame', 'Parent', 'subFrame', 'Child'])

In [175]:
relations.supID

700

In [176]:
fn.frame(700)

frame (700): Committing_crime

[URL] https://framenet2.icsi.berkeley.edu/fnReports/data/frame/Committing_crime.xml

[definition]
  A Perpetrator (generally intentionally) commits a Crime, i.e.
  does something not permitted by the laws of society.  'They
  perpetrated a felony by substituting a lie for negotiations.'
  'The suspect had allegedly committed the crime to gain the
  attention of a female celebrity.'

[semTypes] 0 semantic types

[frameRelations] 12 frame relations
  <Parent=Committing_crime -- Inheritance -> Child=Abusing>
  <Parent=Committing_crime -- Inheritance -> Child=Kidnapping>
  <Parent=Committing_crime -- Inheritance -> Child=Piracy>
  <Parent=Committing_crime -- Inheritance -> Child=Rape>
  <Parent=Committing_crime -- Inheritance -> Child=Robbery>
  <Parent=Committing_crime -- Inheritance -> Child=Smuggling>
  <Parent=Committing_crime -- Inheritance -> Child=Theft>
  <Parent=Misdeed -- Inheritance -> Child=Committing_crime>
  <Complex=Crime_scenario -- Subframe -